#### Gộp Vietnam Airlines và Vietjet Air và Gộp 3 hãng thành 1 


In [98]:
import pandas as pd
import os

# Cài đặt hiển thị đẹp
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)

# Đường dẫn thư mục chứa file
RAW_PATH = "E:\\DATA-PROCESSING\\Du_an_tot_nghiep\\raw_data\\raw_data"

def load_and_prepare(files, drop_airline=False, drop_is_duplicate=False):
    dfs = []
    for file in files:
        df = pd.read_csv(os.path.join(RAW_PATH, file))
        df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
        
        if drop_airline and "airline" in df.columns:
            df.drop(columns="airline", inplace=True)
        if drop_is_duplicate and "is_duplicate" in df.columns:
            df.drop(columns="is_duplicate", inplace=True)
        
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# 1. Vietjet: xoá airline & is_duplicate
df_vietjet = load_and_prepare(["vietjet_1.csv", "vietjet_2.csv"],
                              drop_airline=True, drop_is_duplicate=True)

# 2. Vietnam Airlines: xoá airline
df_vnairlines = load_and_prepare(["VNairlines_1_1.csv", "VNairlines_2_2.csv"],
                                 drop_airline=True)

# 3. Bamboo: giữ nguyên hoàn toàn
df_bamboo = load_and_prepare(["BambooAirway.csv"])

# Gộp tất cả
df_all = pd.concat([df_vietjet, df_vnairlines, df_bamboo], ignore_index=True)

# Xem kết quả
print("Tổng số dòng sau khi gộp:", len(df_all))
print(df_all.head(10))

Tổng số dòng sau khi gộp: 2255788
                     source.name        date    route                        flight_id origin destination fare_class fare_code     price_vnd          taxes departure_time arrival_time booking_class  \
0  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ568-DLIDAD-2025-1-01-1450    DLI         DAD        EL1   QPXVNWS  1.860040e+06  572426.357322            NaN          NaN             Z   
1  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ886-DLIDAD-2025-1-01-1450    DLI         DAD        EC1     ZVNWP  1.929579e+06  577492.077298            NaN          NaN             Z   
2  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ430-DLIDAD-2025-1-01-1450    DLI         DAD        EF1     ZVNWP  2.148786e+06  594070.797220            NaN          NaN             Z   
3  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ180-DLIDAD-2025-1-01-1450    DLI         DAD        DC1     ZVNWP  2.222009e+06  599136.517197       

In [99]:
# Kiểm tra có bao nhiêu hãng bay 
unique_airlines = df_all['operating_airline_name'].dropna().unique()
print("Số lượng hãng bay duy nhất trong 'operating_airline_name':", len(unique_airlines))
print("🛫 Danh sách các hãng bay:")
print(unique_airlines)


Số lượng hãng bay duy nhất trong 'operating_airline_name': 3
🛫 Danh sách các hãng bay:
['Vietjet Air' 'VietNam Airlines' 'BAMBOO AIRWAYS']


### Lọc giữ lại các cột cần thiết 

In [100]:
df_all.drop(columns=['source.name', 'departure_time', 'arrival_time', 'status'], inplace=True)
print("Đã xóa các cột: source.name, departure_time, arrival_time, status khỏi df_all.")


Đã xóa các cột: source.name, departure_time, arrival_time, status khỏi df_all.


In [101]:

print("🧾 Các cột hiện có trong df_all:")
print(df_all.columns.tolist())


🧾 Các cột hiện có trong df_all:
['date', 'route', 'flight_id', 'origin', 'destination', 'fare_class', 'fare_code', 'price_vnd', 'taxes', 'booking_class', 'cabin', 'quota', 'service_code', 'fare_conditions', 'operating_airline_code', 'operating_airline_name']


In [102]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2255788 entries, 0 to 2255787
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   date                    object 
 1   route                   object 
 2   flight_id               object 
 3   origin                  object 
 4   destination             object 
 5   fare_class              object 
 6   fare_code               object 
 7   price_vnd               float64
 8   taxes                   float64
 9   booking_class           object 
 10  cabin                   object 
 11  quota                   float64
 12  service_code            object 
 13  fare_conditions         object 
 14  operating_airline_code  object 
 15  operating_airline_name  object 
dtypes: float64(3), object(13)
memory usage: 275.4+ MB


In [103]:
# Ép kiểu cột date về datetime
df_all['date'] = pd.to_datetime(df_all['date'], errors='coerce')

In [104]:
# Tạo cột giờ
df_all['hour'] = df_all['date'].dt.hour

# Tạo cột "khoảng thời gian bay"
def get_time_slot(hour):
    if 5 <= hour < 12:
        return 'Sáng'
    elif 12 <= hour < 17:
        return 'Chiều'
    elif 17 <= hour < 21:
        return 'Tối'
    else:
        return 'Đêm'

df_all['time_slot'] = df_all['hour'].apply(get_time_slot)

In [105]:
# Đảm bảo date đã đúng kiểu datetime
df_all['date'] = pd.to_datetime(df_all['date'], errors='coerce')

# Thêm các cột thời gian
df_all['month'] = df_all['date'].dt.month
df_all['weekday'] = df_all['date'].dt.day_name()
df_all['week'] = df_all['date'].dt.isocalendar().week

In [106]:
df_all.info()
df_all

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2255788 entries, 0 to 2255787
Data columns (total 21 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   date                    datetime64[ns]
 1   route                   object        
 2   flight_id               object        
 3   origin                  object        
 4   destination             object        
 5   fare_class              object        
 6   fare_code               object        
 7   price_vnd               float64       
 8   taxes                   float64       
 9   booking_class           object        
 10  cabin                   object        
 11  quota                   float64       
 12  service_code            object        
 13  fare_conditions         object        
 14  operating_airline_code  object        
 15  operating_airline_name  object        
 16  hour                    float64       
 17  time_slot               object        
 18  mo

,date,route,flight_id,origin,destination,fare_class,fare_code,price_vnd,taxes,booking_class,cabin,quota,service_code,fare_conditions,operating_airline_code,operating_airline_name,hour,time_slot,month,weekday,week
0,2025-06-13,DLI_DAD,SEG-VJ568-DLIDAD-2025-1-01-1450,DLI,DAD,EL1,QPXVNWS,1.860040e+06,5.724264e+05,Z,ecoPremium,2.0,FBA4-C0FMC2,"PEN19,PEN20,PEN21",VJ,Vietjet Air,0.0,Đêm,6.0,Friday,24
1,2025-06-13,DLI_DAD,SEG-VJ886-DLIDAD-2025-1-01-1450,DLI,DAD,EC1,ZVNWP,1.929579e+06,5.774921e+05,Z,ecoPremium,2.0,FBA4-C0FMC2,"PEN16,PEN17,PEN18",VJ,Vietjet Air,0.0,Đêm,6.0,Friday,24
2,2025-06-13,DLI_DAD,SEG-VJ430-DLIDAD-2025-1-01-1450,DLI,DAD,EF1,ZVNWP,2.148786e+06,5.940708e+05,Z,ecoPremium,2.0,FBA4-C0FMC2,"PEN22,PEN23,PEN24",VJ,Vietjet Air,0.0,Đêm,6.0,Friday,24
3,2025-06-13,DLI_DAD,SEG-VJ180-DLIDAD-2025-1-01-1450,DLI,DAD,DC1,ZVNWP,2.222009e+06,5.991365e+05,Z,ecoPremium,2.0,FBA4-C0FMC2,"PEN28,PEN29,PEN30",VJ,Vietjet Air,0.0,Đêm,6.0,Friday,24
4,2025-06-13,DLI_DAD,SEG-VJ376-DLIDAD-2025-1-01-1450,DLI,DAD,DF1,ZVNWP,2.470690e+06,6.175573e+05,Z,ecoPremium,2.0,FBA4-C0FMC2,"PEN22,PEN23,PEN24",VJ,Vietjet Air,0.0,Đêm,6.0,Friday,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255783,NaT,SGN_HAN,SEG-QH214-SGN_HAN-2026-01-02-1015,SGN,HAN,UEM0FW,EM1,3.372419e+06,1.265544e+06,L,eco,2.0,NaN,"PEN1,PEN2,PEN3",QH,BAMBOO AIRWAYS,NaN,Đêm,NaN,NaN,<NA>
2255784,NaT,SGN_HAN,SEG-QH230-SGN_HAN-2026-01-02-745,SGN,HAN,LES0FW,ES1,2.747938e+06,8.145030e+05,M,eco,6.0,NaN,"PEN1,PEN2,PEN3",QH,BAMBOO AIRWAYS,NaN,Đêm,NaN,NaN,<NA>
2255785,NaT,HAN_DLI,SEG-QH296-HAN_DLI-2026-01-02-1515,HAN,DLI,YEF0FW,ES13,4.557364e+06,1.261177e+06,N,eco,9.0,NaN,"PEN1,PEN2,PEN3",QH,BAMBOO AIRWAYS,NaN,Đêm,NaN,NaN,<NA>
2255786,NaT,HAN_SGN,SEG-QH285-HAN_SGN-2026-01-02-1015,HAN,SGN,WEF0FW,ES1,4.195112e+06,1.224559e+06,W,eco,4.0,NaN,"PEN1,PEN2,PEN3",QH,BAMBOO AIRWAYS,NaN,Đêm,NaN,NaN,<NA>


In [107]:
# Số chuyến bay tổng cộng
print("Tổng số chuyến bay:", len(df_all))

# Số chuyến theo hãng
print("\nSố chuyến theo hãng:")
print(df_all['operating_airline_name'].value_counts())

# Thống kê giá vé
print("\nThống kê giá vé (price_vnd):")
print(df_all['price_vnd'].describe())

Tổng số chuyến bay: 2255788

Số chuyến theo hãng:
operating_airline_name
VietNam Airlines    1164507
Vietjet Air         1067108
BAMBOO AIRWAYS        23790
Name: count, dtype: int64

Thống kê giá vé (price_vnd):
count    2.255405e+06
mean     4.644202e+06
std      3.051185e+06
min      4.107838e+05
25%      2.448000e+06
50%      3.849000e+06
75%      5.996006e+06
max      1.592100e+07
Name: price_vnd, dtype: float64


In [108]:
print("\nGiá vé trung bình theo tháng:")
print(df_all.groupby('month')['price_vnd'].mean().round(0))


Giá vé trung bình theo tháng:
month
1.0     3344166.0
2.0     3995363.0
3.0     3406687.0
4.0     3349370.0
5.0     3333594.0
6.0     2652879.0
7.0     2562875.0
8.0     2564037.0
9.0     2571280.0
10.0    2608576.0
11.0    2665188.0
12.0    2673173.0
Name: price_vnd, dtype: float64


In [109]:
print("\nGiá vé trung bình theo ngày trong tuần:")
print(df_all.groupby('weekday')['price_vnd'].mean().round(0))


Giá vé trung bình theo ngày trong tuần:
weekday
Friday       2833743.0
Monday       2790339.0
Saturday     2788286.0
Sunday       2881056.0
Thursday     2775568.0
Tuesday      2736647.0
Wednesday    2728504.0
Name: price_vnd, dtype: float64


In [110]:
print("\nGiá vé trung bình theo tuần:")
print(df_all.groupby('week')['price_vnd'].mean().round(0))


Giá vé trung bình theo tuần:
week
1     2943236.0
2     3299986.0
3     3376702.0
4     3371631.0
5     3617389.0
6     3925365.0
7     3920435.0
8     4287099.0
9     3997641.0
10    3448884.0
11    3369223.0
12    3372469.0
13    3368340.0
14    3347177.0
15    3346089.0
16    3346985.0
17    3351993.0
18    3355261.0
19    3348176.0
20    3348420.0
21    3332054.0
22    2920749.0
23    2633707.0
24    2690329.0
25    2671333.0
26    2610581.0
27    2574673.0
28    2576106.0
29    2568302.0
30    2555714.0
31    2557170.0
32    2584649.0
33    2524901.0
34    2515611.0
35    2624998.0
36    2663373.0
37    2546230.0
38    2556816.0
39    2543321.0
40    2588707.0
41    2589535.0
42    2602873.0
43    2614807.0
44    2654876.0
45    2658449.0
46    2660212.0
47    2668809.0
48    2656397.0
49    2671626.0
50    2661428.0
51    2673795.0
52    2672254.0
Name: price_vnd, dtype: float64


So sánh mức chênh lệch giá theo hạng vé


In [111]:

# Giả sử df_all là DataFrame đã được gộp sẵn từ các file
# Kiểm tra các cột có liên quan
print(df_all[['route', 'price_vnd', 'cabin', 'operating_airline_name', 'booking_class', 'fare_code', 'fare_conditions']].head())

# 1. So sánh chênh lệch giá theo hạng vé (Economy vs Business)
cabin_avg_price = df_all.groupby('cabin')['price_vnd'].mean().sort_values()
print("\n[1] Giá trung bình theo Cabin:\n", cabin_avg_price.apply(lambda x: f"{x:,.0f} VND"))

# 2. So sánh giá Economy/Business giữa các hãng cùng tuyến
cabin_airline_route = df_all.groupby(['route', 'cabin', 'operating_airline_name'])['price_vnd'].mean().reset_index()
pivot_price_compare = cabin_airline_route.pivot_table(index=['route', 'cabin'], 
                                                       columns='operating_airline_name', 
                                                       values='price_vnd')
print("\n[2] So sánh giá giữa các hãng trên cùng tuyến (theo Cabin):\n", pivot_price_compare.applymap(lambda x: f"{x:,.0f} VND" if pd.notnull(x) else "-"))





     route     price_vnd       cabin operating_airline_name booking_class fare_code    fare_conditions
0  DLI_DAD  1.860040e+06  ecoPremium            Vietjet Air             Z   QPXVNWS  PEN19,PEN20,PEN21
1  DLI_DAD  1.929579e+06  ecoPremium            Vietjet Air             Z     ZVNWP  PEN16,PEN17,PEN18
2  DLI_DAD  2.148786e+06  ecoPremium            Vietjet Air             Z     ZVNWP  PEN22,PEN23,PEN24
3  DLI_DAD  2.222009e+06  ecoPremium            Vietjet Air             Z     ZVNWP  PEN28,PEN29,PEN30
4  DLI_DAD  2.470690e+06  ecoPremium            Vietjet Air             Z     ZVNWP  PEN22,PEN23,PEN24

[1] Giá trung bình theo Cabin:
 cabin
eco           3,175,016 VND
ecoPremium    4,384,022 VND
business      6,632,326 VND
Name: price_vnd, dtype: object

[2] So sánh giá giữa các hãng trên cùng tuyến (theo Cabin):
 operating_airline_name BAMBOO AIRWAYS VietNam Airlines    Vietjet Air
route   cabin                                                        
CXR_DAD business          

C:\Users\datde\AppData\Local\Temp\ipykernel_24620\273469221.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print("\n[2] So sánh giá giữa các hãng trên cùng tuyến (theo Cabin):\n", pivot_price_compare.applymap(lambda x: f"{x:,.0f} VND" if pd.notnull(x) else "-"))


In [112]:
# 3. Xem hãng nào giá tốt hơn cho từng hạng
def better_price(row):
    values = row.dropna()
    if len(values) >= 2:
        return values.idxmin()
    return None

pivot_price_compare['Best_price_airline'] = pivot_price_compare.apply(better_price, axis=1)
print("\n[3] Hãng có giá tốt nhất theo từng tuyến và hạng:\n", pivot_price_compare[['Best_price_airline']])

# 4. Phân tích độ linh hoạt (Booking_Class, Fare_Code, Fare_Conditions)
flex_analysis = df_all.groupby(['booking_class', 'fare_code', 'fare_conditions'])['price_vnd'].agg(['mean', 'count']).sort_values(by='mean')
print("\n[4] Phân tích độ linh hoạt:\n", flex_analysis.head(10))  # hiển thị 10 dòng đầu


[3] Hãng có giá tốt nhất theo từng tuyến và hạng:
 operating_airline_name Best_price_airline
route   cabin                            
CXR_DAD business              Vietjet Air
        eco                   Vietjet Air
        ecoPremium            Vietjet Air
CXR_DLI business              Vietjet Air
        eco                   Vietjet Air
...                                   ...
SGN_HAN ecoPremium            Vietjet Air
SGN_UIH business                     None
        eco                          None
UIH_SGN business                     None
        eco                          None

[124 rows x 1 columns]

[4] Phân tích độ linh hoạt:
                                                  mean  count
booking_class fare_code fare_conditions                     
A             AAP4VNWP  PEN1,PEN2,PEN3   6.988588e+05      8
P             PAP2VNWS  PEN1,PEN2,PEN3   9.341347e+05      7
R             RAP5VNWS  PEN1,PEN2,PEN3   1.122592e+06     16
P             PAP4VNWP  PEN1,PEN2,PEN3   1.

Những ngày có nhiều chuyến bay nhất

In [113]:
# Đảm bảo cột ngày là dạng datetime
df_all['date'] = pd.to_datetime(df_all['date'])

# Đếm số chuyến bay theo từng ngày
flights_per_day = df_all['date'].value_counts().sort_values(ascending=False)

# Hiển thị top 10 ngày có nhiều chuyến nhất
print("Top 10 ngày có nhiều chuyến bay nhất:")
print(flights_per_day.head(10))


Top 10 ngày có nhiều chuyến bay nhất:
date
2025-08-17    4829
2025-08-13    4774
2025-08-10    4752
2025-07-20    4745
2025-07-30    4738
2025-07-27    4720
2025-07-23    4718
2025-08-24    4688
2025-07-29    4684
2025-07-06    4675
Name: count, dtype: int64


In [114]:
# Trung bình giá vé theo tháng
month_avg_price = df_all.groupby('month')['price_vnd'].mean().sort_values(ascending=False)

print("\n[3] Giá vé trung bình theo tháng (giảm dần):\n")
print(month_avg_price.apply(lambda x: f"{x:,.0f} VND"))

# Tháng có giá vé trung bình cao nhất
max_month = month_avg_price.idxmax()
max_price = month_avg_price.max()

print(f"\nTháng có giá vé trung bình cao nhất: Tháng {max_month} với mức giá trung bình: {max_price:,.0f} VND")



[3] Giá vé trung bình theo tháng (giảm dần):

month
2.0     3,995,363 VND
3.0     3,406,687 VND
4.0     3,349,370 VND
1.0     3,344,166 VND
5.0     3,333,594 VND
12.0    2,673,173 VND
11.0    2,665,188 VND
6.0     2,652,879 VND
10.0    2,608,576 VND
9.0     2,571,280 VND
8.0     2,564,037 VND
7.0     2,562,875 VND
Name: price_vnd, dtype: object

Tháng có giá vé trung bình cao nhất: Tháng 2.0 với mức giá trung bình: 3,995,363 VND


In [115]:
# Trung bình giá vé theo tháng và hãng
monthly_airline_price = df_all.groupby(['operating_airline_name', 'month'])['price_vnd'].mean()

# Tìm tháng có giá cao nhất cho mỗi hãng
highest_month_per_airline = monthly_airline_price.groupby('operating_airline_name').idxmax()
highest_price_per_airline = monthly_airline_price.groupby('operating_airline_name').max()

# Gộp kết quả
result = pd.DataFrame({
    'Tháng cao nhất': highest_month_per_airline.map(lambda x: x[1]),
    'Giá trung bình cao nhất (VND)': highest_price_per_airline
})
result['Giá trung bình cao nhất (VND)'] = result['Giá trung bình cao nhất (VND)'].apply(lambda x: f"{x:,.0f}")
print("\nTháng có giá vé trung bình cao nhất theo từng hãng:\n")
print(result)



Tháng có giá vé trung bình cao nhất theo từng hãng:

                        Tháng cao nhất Giá trung bình cao nhất (VND)
operating_airline_name                                              
Vietjet Air                        2.0                     3,995,363


Trung bình giá vé theo hãng, tìm hãng có giá vé trung bình thấp nhất

In [116]:
# Tính trung bình giá vé theo hãng bay
avg_price_by_airline = df_all.groupby("operating_airline_name")["price_vnd"].mean().sort_values()

# Hiển thị hãng có giá vé trung bình rẻ nhất
print("Hãng có giá vé trung bình rẻ nhất:")
print(avg_price_by_airline.head(1).apply(lambda x: f"{x:,.0f} VND"))

# Hiển thị toàn bộ để so sánh
print("\nGiá vé trung bình theo từng hãng:")
print(avg_price_by_airline.apply(lambda x: f"{x:,.0f} VND"))


Hãng có giá vé trung bình rẻ nhất:
operating_airline_name
Vietjet Air    2,792,002 VND
Name: price_vnd, dtype: object

Giá vé trung bình theo từng hãng:
operating_airline_name
Vietjet Air         2,792,002 VND
BAMBOO AIRWAYS      4,074,105 VND
VietNam Airlines    6,353,131 VND
Name: price_vnd, dtype: object


Tác động của Booking Class và Fare Code đến giá

In [117]:
avg_prices = df_all.groupby(['booking_class', 'fare_code'])["price_vnd"].mean().sort_values()
print(avg_prices.apply(lambda x: f"{x:,.0f} VND"))

booking_class  fare_code
A              AAP4VNWP       698,859 VND
P              PAP2VNWS       934,135 VND
               PAP4VNWP     1,207,625 VND
Q              QAP6VNWS     1,239,251 VND
P              PPXVNWS      1,242,406 VND
                                ...      
J              IVNW         8,754,114 VND
Z              JVNW         8,765,676 VND
C              JVNWP        8,930,066 VND
               JVNW         9,097,492 VND
J              CVNW         9,272,827 VND
Name: price_vnd, Length: 851, dtype: object


In [118]:
#%%
# tổng các chuyến bay theo HÀNH TRÌNH
print(df_all['route'].unique())

route_counts = df_all['route'].value_counts().reset_index()
route_counts.columns = ['route', 'flight_count']

print("Tổng các chuyến bay theo HÀNH TRÌNH:")

route_counts['flight_count'] = route_counts['flight_count'].astype(str) + ' chuyến'

print(route_counts[['route', 'flight_count']])
print(f"Tuyến nhiều lượt bay nhất: {route_counts['flight_count'].max()}")
print(f"Tuyến ít lượt bay nhất: {route_counts['flight_count'].min()}")

['DLI_DAD' 'DLI_HAN' 'DLI_HPH' 'DLI_SGN' 'HAN_CXR' 'HAN_DAD' 'HAN_DLI'
 'HAN_SGN' 'HPH_CXR' 'HPH_DAD' 'HPH_DLI' 'HPH_HAN' 'HPH_HUI' 'HPH_SGN'
 'HUI_DAD' 'HUI_DLI' 'HUI_HAN' 'HUI_SGN' 'SGN_CXR' 'SGN_DAD' 'SGN_DLI'
 'SGN_HAN' 'CXR_DLI' 'CXR_DAD' 'CXR_HAN' 'CXR_SGN' 'DAD_CXR' 'DAD_DLI'
 'DAD_HAN' 'DAD_SGN' 'DLI_CXR' 'DLI_PQC' nan 'PQC_CXR' 'PQC_DAD' 'PQC_DLI'
 'PQC_HAN' 'PQC_HPH' 'PQC_HUI' 'PQC_SGN' 'CXR_PQC' 'SGN_UIH' 'UIH_SGN']
Tổng các chuyến bay theo HÀNH TRÌNH:
      route   flight_count
0   CXR_HAN  181278 chuyến
1   HAN_DAD  176837 chuyến
2   HAN_CXR  175482 chuyến
3   SGN_DAD  174326 chuyến
4   DAD_SGN  172772 chuyến
5   DAD_HAN  165778 chuyến
6   DAD_CXR  139266 chuyến
7   SGN_CXR  125145 chuyến
8   CXR_DAD  123062 chuyến
9   CXR_SGN  113524 chuyến
10  SGN_HAN   88917 chuyến
11  HAN_SGN   79281 chuyến
12  HUI_SGN   39621 chuyến
13  DLI_SGN   34709 chuyến
14  HPH_SGN   33287 chuyến
15  HUI_HAN   31666 chuyến
16  HAN_DLI   30093 chuyến
17  DLI_HAN   28013 chuyến
18  SGN_DLI   27343

In [119]:
route_price_avg = df_all.groupby('route')['price_vnd'].mean().reset_index()

min_price_row = route_price_avg.loc[route_price_avg['price_vnd'].idxmin()]

max_price_row = route_price_avg.loc[route_price_avg['price_vnd'].idxmax()]

print(f"Tuyến bay có giá vé trung bình rẻ nhất: {min_price_row['route']} với giá {min_price_row['price_vnd']:.2f} VND")
print(f"Tuyến bay có giá vé trung bình cao nhất: {max_price_row['route']} với giá {max_price_row['price_vnd']:.2f} VND")

Tuyến bay có giá vé trung bình rẻ nhất: SGN_UIH với giá 3513301.26 VND
Tuyến bay có giá vé trung bình cao nhất: PQC_HAN với giá 6983807.68 VND


In [120]:
#%%
# Tính giá vé trung bình theo từng tuyến
route_price_avg = df_all.groupby('route')['price_vnd'].mean().reset_index()

# Sắp xếp giá vé trung bình giảm dần
top_highest = route_price_avg.sort_values(by='price_vnd', ascending=False).head(5)

# Sắp xếp giá vé trung bình tăng dần
top_lowest = route_price_avg.sort_values(by='price_vnd', ascending=True).head(5)

print("Top 5 tuyến bay có giá vé trung bình cao nhất:")
print(top_highest)

print("\nTop 5 tuyến bay có giá vé trung bình thấp nhất:")
print(top_lowest)

Top 5 tuyến bay có giá vé trung bình cao nhất:
      route     price_vnd
32  PQC_HAN  6.983808e+06
33  PQC_HPH  6.207264e+06
29  PQC_CXR  5.969846e+06
35  PQC_SGN  5.770892e+06
30  PQC_DAD  5.726581e+06

Top 5 tuyến bay có giá vé trung bình thấp nhất:
      route     price_vnd
40  SGN_UIH  3.513301e+06
18  HAN_SGN  3.555289e+06
39  SGN_HAN  3.561754e+06
24  HPH_SGN  3.578580e+06
6   DAD_DLI  3.595977e+06


In [121]:
#%%

df_all['date'] = pd.to_datetime(df_all['date'], errors='coerce')

# Danh sách ngày lễ lớn Việt Nam
holidays = [
    '2025-01-01',  # Tết Dương lịch
    '2025-02-10',  # 29 Tết
    '2025-02-11',  # 30 Tết
    '2025-02-12',  # Mùng 1
    '2025-02-13',  # Mùng 2
    '2025-02-14',  # Mùng 3
    '2025-02-15',  # Mùng 4
    '2025-02-16',  # Mùng 5
    '2025-04-30',  # Giải phóng miền Nam
    '2025-05-01',  # Quốc tế Lao động
    '2025-09-02'   # Quốc khánh
]

holidays = pd.to_datetime(holidays)

df_all['tag_ngay'] = df_all['date'].apply(lambda x: 'Ngày lễ' if x in holidays else 'Ngày bình thường')

le_df = df_all[df_all['tag_ngay'] == 'Ngày lễ']
print(le_df[['date', 'tag_ngay']])


              date tag_ngay
3772    2025-09-02  Ngày lễ
3773    2025-09-02  Ngày lễ
3774    2025-09-02  Ngày lễ
3775    2025-09-02  Ngày lễ
3776    2025-09-02  Ngày lễ
...            ...      ...
1058539 2025-09-02  Ngày lễ
1058540 2025-09-02  Ngày lễ
1058541 2025-09-02  Ngày lễ
1058542 2025-09-02  Ngày lễ
1058543 2025-09-02  Ngày lễ

[3267 rows x 2 columns]


In [122]:
#%%
# Mô tả sân bay
airport_dict = {
    'DAD': 'Sân bay quốc tế Đà Nẵng',
    'HAN': 'Sân bay quốc tế Nội Bài',
    'HPH': 'Sân bay quốc tế Cát Bi',
    'SGN': 'Sân bay quốc tế Tân Sơn Nhất',
    'CXR': 'Sân bay quốc tế Cam Ranh',
    'DLI': 'Sân bay Liên Khương',
    'HUI': 'Sân bay Phú Bài',
    'PQC': 'Sân bay quốc tế Phú Quốc',
    'UIH': 'Sân bay Phù Cát'
}

# Hàm chuyển đổi route thành tên đầy đủ
def replace_route_with_names(route):
    try:
        dep, arr = route.split('_')
        dep_name = airport_dict.get(dep, dep)
        arr_name = airport_dict.get(arr, arr)
        return f"{dep_name} - {arr_name}"
    except:
        return route  # Giữ nguyên nếu có lỗi

df_all['route_full_name'] = df_all['route'].apply(replace_route_with_names)

# Kiểm tra kết quả
print(df_all[['route', 'route_full_name']].head(10))

df_all.rename(columns={'route_full_name': 'description'}, inplace=True)

     route                                route_full_name
0  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng
1  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng
2  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng
3  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng
4  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng
5  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng
6  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng
7  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng
8  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng
9  DLI_DAD  Sân bay Liên Khương - Sân bay quốc tế Đà Nẵng


In [ ]:
# Tính giá gốc (net_price) và tỷ lệ thuế trên tổng giá
df_all['net_price'] = df_all['price_vnd'] - df_all['taxes']
df_all['tax_ratio'] = df_all['taxes'] / df_all['price_vnd']
display(df_all[['price_vnd', 'taxes', 'net_price', 'tax_ratio']].sample(5))


,price_vnd,taxes,net_price,tax_ratio
23961,2.843711e+06,6.451885e+05,2.198522e+06,0.226883
1057714,1.840294e+06,5.175449e+05,1.322749e+06,0.281229
1310549,3.566000e+06,1.208000e+06,2.358000e+06,0.338755
2076948,7.525000e+06,1.501000e+06,6.024000e+06,0.199468
877867,1.903321e+06,5.223930e+05,1.380928e+06,0.274464


In [124]:
#Phân loại vé thành Economy / Premium / Business
def infer_fare_type(row):
    code = str(row['fare_code']).upper()
    cond = str(row['fare_conditions']).upper()
    if any(k in cond for k in ['BUS', 'PRE33', 'PEN33']) or code.startswith('B'):
        return 'Business'
    elif any(k in cond for k in ['PRE', 'FLE']) or code.startswith('E'):
        return 'Premium'
    elif any(k in cond for k in ['PEN', 'ECO']) or code.startswith('Y') or code.startswith('Z'):
        return 'Economy'
    else:
        return 'Không rõ'

df_all['fare_type'] = df_all.apply(infer_fare_type, axis=1)
display(df_all[['fare_code', 'fare_conditions', 'fare_type']].sample(10))



,fare_code,fare_conditions,fare_type
65060,JVNW,"PEN4,PEN5,PEN6",Economy
71315,CVNW,"PEN55,PEN56,PEN57",Economy
573916,JVNW,"PEN7,PEN8,PEN9",Economy
1297785,WVNW,"PEN4,PEN5,PEN6",Economy
510470,ZVNWS,"PEN7,PEN8,PEN9",Economy
2237999,BS1,"PEN16,PEN17,PEN18",Business
1466797,BVNW,"PEN4,PEN5,PEN6",Business
994215,MVNWS,"PEN7,PEN8,PEN9",Economy
1394606,WVNWP,"PEN25,PEN26,PEN27",Economy
1871131,IAPVNWP,"PEN37,PEN38,PEN39",Economy


In [125]:
#Tuyến có mức thuế trung bình cao nhất
avg_tax_by_route = df_all.groupby('route')['taxes'].mean().reset_index().sort_values(by='taxes', ascending=False)
print ("Tuyến có mức thuế trung bình cao nhất: \n")
print(avg_tax_by_route)

Tuyến có mức thuế trung bình cao nhất: 

      route         taxes
33  PQC_HPH  1.403582e+06
29  PQC_CXR  1.385990e+06
32  PQC_HAN  1.380902e+06
30  PQC_DAD  1.367926e+06
34  PQC_HUI  1.343329e+06
13  DLI_PQC  1.323759e+06
3   CXR_PQC  1.307894e+06
31  PQC_DLI  1.304514e+06
35  PQC_SGN  1.149195e+06
22  HPH_HAN  1.093103e+06
36  SGN_CXR  1.069033e+06
41  UIH_SGN  1.054320e+06
23  HPH_HUI  1.033791e+06
4   CXR_SGN  1.021677e+06
15  HAN_CXR  1.011698e+06
20  HPH_DAD  1.011486e+06
12  DLI_HPH  1.010369e+06
21  HPH_DLI  1.010043e+06
40  SGN_UIH  1.007863e+06
25  HUI_DAD  1.007255e+06
9   DLI_CXR  1.003030e+06
19  HPH_CXR  1.001265e+06
2   CXR_HAN  9.869561e+05
26  HUI_DLI  9.834662e+05
5   DAD_CXR  9.813780e+05
0   CXR_DAD  9.778260e+05
37  SGN_DAD  9.730663e+05
17  HAN_DLI  9.486907e+05
16  HAN_DAD  9.428774e+05
11  DLI_HAN  9.318559e+05
1   CXR_DLI  9.281069e+05
8   DAD_SGN  9.125822e+05
10  DLI_DAD  9.119439e+05
27  HUI_HAN  9.044377e+05
7   DAD_HAN  9.044208e+05
6   DAD_DLI  8.826428e+

In [ ]:
##Tỷ lệ từng fare_type trong mỗi hãng
fare_type_ratio = df_all.groupby(['operating_airline_name', 'fare_type'])['price_vnd'].count().reset_index()
fare_type_ratio = fare_type_ratio.pivot(index='operating_airline_name', columns='fare_type', values='price_vnd').fillna(0)
fare_type_ratio = fare_type_ratio.div(fare_type_ratio.sum(axis=1), axis=0)
print('Tỷ lệ từng loại vé mỗi hãng: \n')
fare_type_ratio

Tỷ lệ từng loại vé mỗi hãng: 



fare_type,Business,Economy,Premium
operating_airline_name,,,
BAMBOO AIRWAYS,0.255233,0.000000,0.744767
VietNam Airlines,0.142043,0.844840,0.013117
Vietjet Air,0.144628,0.842191,0.013180


In [ ]:
#Giá cao nhất theo loại vé của từng hãng
max_price_by_faretype = df_all.groupby(['operating_airline_name', 'fare_type'])['price_vnd'].max().reset_index()
max_price_by_faretype

,operating_airline_name,fare_type,price_vnd
0,BAMBOO AIRWAYS,Business,1.174000e+07
1,BAMBOO AIRWAYS,Premium,7.384000e+06
2,VietNam Airlines,Business,1.581300e+07
3,VietNam Airlines,Economy,1.592100e+07
4,VietNam Airlines,Premium,6.210000e+06
5,Vietjet Air,Business,7.282203e+06
6,Vietjet Air,Economy,7.282203e+06
7,Vietjet Air,Premium,2.741476e+06


In [128]:
price_std = df_all.groupby('route')['price_vnd'].std()
df_all['price_std'] = df_all['route'].map(price_std)

# Gán nhãn mức biến động (cao / trung bình / thấp)
df_all['price_volatility'] = pd.qcut(df_all['price_std'], q=3, labels=['Ổn định', 'Trung bình', 'Biến động cao'])

display(df_all[['route', 'price_vnd', 'price_std', 'price_volatility']].sample(5))


,route,price_vnd,price_std,price_volatility
65053,HAN_CXR,2.992919e+06,2.889280e+06,Trung bình
1165708,HAN_CXR,5.362000e+06,2.889280e+06,Trung bình
1492010,PQC_DLI,2.832000e+06,1.828209e+06,Ổn định
1660057,SGN_DAD,7.525000e+06,3.337649e+06,Biến động cao
198622,HAN_DAD,2.064972e+06,3.285292e+06,Trung bình
